In [1]:
import numpy as np
import matplotlib.pyplot as plt
import control
import math
import cmath

Considere o sistema apresentado na Fig. 3, sendo o período de amostragem de 0,15
s e a função de transferência discreta do controlador digital é dada por: 

Gc(z) = Kc * (z - alfa)/(z - beta)

Determine os parâmetros de β e Kc do controlador digital de maneira que o sistema
apresente polos dominantes de segunda ordem em malha fechada que tenham um fator
de amortecimento ζ = 0,6 e uma frequência natural ωn = 5 rad/s. (demonstre, prove
matematicamente). Considere que o zero do controlador cancelará o polo da função de
transferência da planta que não está em z = 1.

In [2]:
T= 0.15 #periodo de amostragem

G = control.tf([1],[1,5])
print(G)
Gz = control.c2d(G,T) #passando para o dominio discreto (dominio z)
print(Gz)



  1
-----
s + 5


  0.1055
----------
z - 0.4724

dt = 0.15



In [3]:
H = control.tf([1],[1,0])
print(H)
GH = H * G
GHz = control.c2d(GH, T)
print(GHz)


1
-
s


0.008895 z + 0.006934
----------------------
z^2 - 1.472 z + 0.4724

dt = 0.15



In [12]:
#para os valores abaixo de zeta e wn abaixo
zeta = 0.6
wn = 5

#calcular o valor do polo z1 desejado utilizando o zeta e wn informados
z1_mod = math.e**(-T *zeta*wn)
print("|z1| = ", z1_mod)
z1_ang = T*wn*np.sqrt(1-(zeta**2))
z1_ang_deg = math.degrees(z1_ang)
print("z1< = ", z1_ang_deg)

|z1| =  0.6376281516217733
z1< =  34.3774677078494


In [9]:
z_rect = cmath.rect(z1_mod, z1_ang)
print(z_rect)

(0.5262572226024779+0.3600319366380228j)


In [14]:
#Para o sistema ser estavel estável precisamos que a soma dos angulos seja mais ou menos igual a 180 graus.
angle = np.pi - cmath.phase(GHz(z_rect))
print(math.degrees(angle)) #angulo do controlador 

28.83938481284644


In [17]:
beta = (z_rect.imag / math.tan(-angle)) - z_rect.real #formula pra quando eh (z+beta)
print(beta)

-1.1800878721809764


In [20]:
controler = control.tf([1,-0.4724],[1,+beta],T) #calculo do controlador. O valor do numerador é [z,alfa], sendo alfa o valor que cancela o polo do numerador.
print(controler)


z - 0.4724
----------
 z - 1.18

dt = 0.15



In [21]:
FTMA = controler * GHz 

In [23]:
Kc = 1/np.abs(FTMA(z_rect))
print(Kc)

36.86197673135336


In [26]:
Output = controler * Kc * GHz

In [27]:
print(Output)


 0.3279 z^2 + 0.1007 z - 0.1208
---------------------------------
z^3 - 2.652 z^2 + 2.21 z - 0.5574

dt = 0.15

